<a href="https://colab.research.google.com/github/nrjcs/swym/blob/master/swym_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import
import os;
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.metrics import categorical_crossentropy
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from numpy import *
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib

K.set_image_data_format('channels_first') # set format

In [0]:
#intialization
train_labels=[]
train_samples=[]


In [0]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
# initialization
img_rows, img_cols=256, 256
img_channels=1
ip_path='/content/drive/My Drive/img_data/'
op_path='/content/drive/My Drive/img_data_updated/'

In [0]:
#getting list of images
listing=os.listdir(ip_path) #list of the names of the entries in the arg (directory)
num_samples=size(listing)
print(num_samples)


In [0]:
#image preprocessing...resize...convert to grayscale
for file in listing:
    im=Image.open(ip_path+'/'+file)
    img=im.resize((img_rows,img_cols))
    gray=img.convert('L')
    gray.save(op_path+'/'+file, "JPEG")
imlist=os.listdir(op_path)  
imlist.sort()

In [0]:
# viewing images
im1=array(Image.open('/content/drive/My Drive/img_data_updated/'+imlist[0]))
print(im1)
plt.imshow(im1)
#plt.imshow(im1,cmap='gray')
print(im1.shape)
m,n=im1.shape[0:2]
print m,m

In [0]:
#flatten all the images into one matrix
immat=array([array(Image.open('/content/drive/My Drive/img_data_updated/'+imseq)).flatten() 
           for imseq in imlist],'f')

#print immat
#print immat.shape

In [0]:
# initialize labels
labels=np.ones((num_samples),dtype=int)
print labels

In [0]:
#reset labels
labels[0:100]=0
labels[100:200]=1
print labels

In [0]:
#make sample label pairs
data,label=shuffle(immat,labels,random_state=2)
for i in range(200):
   print data[i],label[i]

In [0]:
#combine data and labels as single input

#this step in not necessary as we can input the data and label as two arrarys
train_data=[data,label]
#print train_data[0].shape #shape of samples

#print train_data[1].shape #shape of labels
#print train_data

In [0]:
#check the images from the flattened matrix
check_img=immat[104].reshape(img_rows,img_cols)
plt.imshow(check_img)
plt.imshow(check_img,cmap='gray')

In [0]:
#model parameters

batch_size=20
n_classes=2
n_epochs=2
img_channels=1 # no. of imagae channels
n_filters=32 #no. of conv filters to use
n_pool=2  #size of area for max pooling
n_conv=3  #size of conv kernel

(X,Y)=(train_data[0],train_data[1])  #this is not necessary data can be passed directly as samples and labels
print (X,Y)


In [0]:
# preprocessing
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=4)
X_train=X_train.reshape(X_train.shape[0],1,img_rows,img_cols)
print X_train.shape
X_test=X_test.reshape(X_test.shape[0],1,img_rows,img_cols)
print X_test.shape

X_train/=255
X_test/=255

# following steps make one hot matrix used by the model to fit data
print Y_train.shape   #current shape
print Y_test.shape

from keras.utils import np_utils
Y_train=np_utils.to_categorical(Y_train,n_classes)
Y_test=np_utils.to_categorical(Y_test,n_classes)
#print Y_train.shape
#print Y_test.shape
#print Y_test

In [0]:
# test the modifations of data and lables

plt.imshow(X_train[170,0],interpolation='nearest',cmap='gray')
#print("label:",Y_train[170,:])

In [0]:
# deifne the model layout
model=Sequential()  #type of model

#add layers to the model

model.add(Convolution2D(n_filters,n_conv,n_conv,border_mode='valid',input_shape=(1,img_rows,img_cols)))
convol1=Activation('relu')
model.add(convol1)
model.add(Convolution2D(n_filters,n_conv,n_conv))
convol2=Activation('relu')
model.add(convol2)
model.add(MaxPooling2D(pool_size=(n_pool,n_pool)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(n_classes))
model.add(Activation('softmax'))



In [0]:
# compile the model

model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

# print the model summary

model.summary()

In [0]:
# train the model

trainin=model.fit(X_train,Y_train,batch_size=batch_size,epochs=n_epochs,verbose=1,validation_data=(X_test,Y_test))

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])